# **Security Course**
### <span style="font-weight:bold;">Instructor:</span> <span style="font-size:19px;"><a href="https://ir.linkedin.com/in/hrshahriari">Dr. Hamid Reza Shahriari</a></span>
### <span style="font-weight:bold;">Organization</span>
### <span style="font-size:19px;"><a href="https://aut.ac.ir/en">Amirkabir University of Technology (Tehran Polytechnic)</a></span>
<img src="../assets/AKUT-white-logo.png" alt="Amirkabir University of Technology logo" style="width:300px;">

# **DES Encryption Algorithm**
In this notebook, we will walk through the steps of implementing the Data Encryption Standard (DES) algorithm.

### Imports

In [336]:
%run ./utils.ipynb

### Key Scheduler

In [337]:
def key_scheduler(key: str)-> list[str]:
    """_summary_

    Args:
        key (str): input key

    Returns:
        list[str]: list of the subkeys for each round
    """
    subkey_list = []
    permuted_key= permuted_choice_1(key)

    R = permuted_key[28:]
    L = permuted_key[:28]

    for i in range(16):
        R=left_shit_register(R, SHIFT_TABLE[i])
        L=left_shit_register(L, SHIFT_TABLE[i])

        subkey=permuted_choice_2(L+R)
        
        subkey_list.append(subkey)

    return subkey_list 

### Round Function

In [338]:
def round_fucntion(R: str, subkey: str) -> str:
    """
    _summary_

    Args:
        R (str): lowest 32 bits of round input
        subkey (str): subkey that belong to this round

    Returns:
        str: round output
    """
    round_function_initial_permutation_output = round_function_initial_expansion_permutation(R)
    xor_output= XOR(round_function_initial_permutation_output, subkey)
    sbox_output= sbox_operation(xor_output)
    return round_function_final_permutation(sbox_output)

### DES Algorithm

In [339]:
def DES(binary_text: str, key: str, is_decipher= False) -> str:
    """_summary_

    Args:
        binary_plain_text (str): _description_
        key (str): _description_

    Returns:
        str: _description_
    """
    if len(binary_text)!=64:
        raise Exception(f'length of the binary text is not equal to 64 bit!. (length= {len(binary_text)})')
    if len(key)!=64:
        raise Exception(f'length of the key is not equal to 64 bit!. (length= {len(key)})')
    subkey_list= key_scheduler(key)
    if is_decipher:
        subkey_list[::-1]
    
    initial_permutation_output= initial_permutation(binary_text)
    for i in range(16):
        R=initial_permutation_output[32:]
        L=initial_permutation_output[:32]
        round_fucntion_output= round_fucntion(R, subkey_list[i])
        temp=XOR(L, round_fucntion_output)
        L=temp
        if (i!=15):
            L, R = R, L
    return final_permutation(L+R)

### Testing Implementated DES Algorithm

In [340]:
key=string_to_binary("ENCRYPT!")
if len(key)!=64:
    raise Exception(f'length of the key is not equal to 64 bit!. (length= {len(key)})')

1- encryption

In [341]:
binary_text = string_to_binary("SECURITY")
if len(binary_text)!=64:
    raise Exception(f'length of the binary plain text is not equal to 64 bit!. (length= {len(binary_text)})')

DES_cipher_output= DES(binary_text, key)
print(DES_cipher_output)

0101001101000100000001100001010000000011000010000001010000011100


2-decryption

In [342]:
binary_cipher_text = DES_cipher_output[32:]+DES_cipher_output[:32]
binary_cipher_text = DES_cipher_output
if len(binary_cipher_text)!=64:
    raise Exception(f'length of the binary cipher text is not equal to 64 bit!. (length= {len(binary_text)})')

DES_decipher_output= DES(binary_cipher_text, key, is_decipher= True)
binary_to_string(DES_decipher_output)

'SECURITY'

# **Attachment: 3DES Encryption Algorithm**

In [343]:
def three_DES(binary_text: str, key1: str, key2: str, key3: str, is_decipher=False) -> str:
    
    is_decipher_1=False
    is_decipher_2=True
    is_decipher_3=False
    if is_decipher:
        is_decipher_1= not is_decipher_1
        is_decipher_2= not is_decipher_2
        is_decipher_3= not is_decipher_3

    output_1= DES(binary_text, key1, is_decipher_1)
    output_2= DES(output_1, key2, is_decipher_2)
    output_3= DES(output_2, key3, is_decipher_3)
    return output_3

### Testing Implementated 3DES Algorithm

In [344]:
key1=string_to_binary("ENCRYPT1")
key2=string_to_binary("ENCRYPT2")
key3=string_to_binary("ENCRYPT3")

if len(key1)!=64:
    raise Exception(f'length of the key1 is not equal to 64 bit!. (length= {len(key1)})')
if len(key2)!=64:
    raise Exception(f'length of the key2 is not equal to 64 bit!. (length= {len(key2)})')
if len(key3)!=64:
    raise Exception(f'length of the key3 is not equal to 64 bit!. (length= {len(key3)})')

1-encryption

In [345]:
binary_text = string_to_binary("SECURITY")
if len(binary_text)!=64:
    raise Exception(f'length of the binary plain text is not equal to 64 bit!. (length= {len(binary_text)})')

three_DES_cipher_output= three_DES(binary_text, key1, key2, key3)
print(three_DES_cipher_output)

0101001101000100000001100001010000000011000010000001010000011100


2-decryption

In [347]:
binary_cipher_text = DES_cipher_output
if len(binary_cipher_text)!=64:
    raise Exception(f'length of the binary cipher text is not equal to 64 bit!. (length= {len(binary_text)})')

DES_decipher_output= three_DES(binary_cipher_text, key1, key2, key3, is_decipher= True)
binary_to_string(DES_decipher_output)

'SECURITY'